In [1]:
# Basic operating system (os), numerical, and plotting functionality
import os
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from random import randint
#import gradio as gr
import argparse
import cv2
import tensorflow as tf

# import emnist
from emnist import list_datasets
from emnist import extract_training_samples
from emnist import extract_test_samples

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages

In [0]:
#from sklearn.utils import shuffle

# Feature extractors and classification algorithm
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier

# scikit-learn performance metric utilities
#from sklearn.metrics import confusion_matrix
#from sklearn.metrics import accuracy_score
#import seaborn as sns

In [3]:
list_datasets()

['balanced', 'byclass', 'bymerge', 'digits', 'letters', 'mnist']

In [17]:
# get training and testing data and labels from the emnist balanced datset
training_images, training_labels = extract_training_samples("balanced")
testing_images, testing_labels = extract_test_samples("balanced")

In [3]:
# letters in the correct order so that they can be indexed using the labels
class_mapping = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabdefghnqrt'

In [11]:
# expand new (channel) axis
(training_images, training_labels), (testing_images, testing_labels) = tf.keras.datasets.mnist.load_data()
training_images = np.expand_dims(training_images, axis=-1)
training_images = np.repeat(training_images, 3, axis = -1)

# normalize data
training_images = training_images.astype('float32')/255

# resize input shape to be 32x32 (minimum size for resnet50)
training_images = tf.image.resize(training_images, [32, 32])

# one hot labels
training_labels = tf.keras.utils.to_categorical(training_labels, num_classes=10)

In [12]:
# expand new (channel) axis
testing_images = np.expand_dims(testing_images, axis=-1)
testing_images = np.repeat(testing_images, 3, axis = -1)

# normalize data
testing_images = testing_images.astype('float32')/255

# resize input shape to be 32x32 (minimum size for resnet50)
testing_images = tf.image.resize(testing_images, [32, 32])

# one hot labels
testing_labels = tf.keras.utils.to_categorical(testing_labels, num_classes=10)

In [14]:
# augment data using ImageDatagenerator to shift the data in different manners
aug = ImageDataGenerator(rotation_range = 10, zoom_range=0.05, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, horizontal_flip=False, fill_mode="nearest")

In [13]:
# create and compile resnet network
input = tf.keras.Input(shape=(32, 32, 3))
model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights="imagenet", input_tensor=input)
# apply global max pooling
pool = tf.keras.layers.GlobalMaxPooling2D()(model.output)

# add classification layer
classification = tf.keras.layers.Dense(10, activation='softmax', use_bias=True)(pool)

# bind all layers
resnet_model = tf.keras.Model(model.input, classification)

In [14]:
training_images.shape

TensorShape([60000, 32, 32, 3])

In [15]:
# train network 
resnet_model.compile(loss="categorical_crossentropy", optimizer="Adam",
	metrics=["accuracy"])
resnet_model.fit(training_images, training_labels, batch_size=10000, epochs=4)

Epoch 1/4


In [0]:
# evaluate network
predictions = model.predict(testing_images)
print(classification_report(testing_labels.argmax(axis=1), predictions.argmax(axis=1), target_names=class_mapping))